In [1]:
#RadioML(DeepSig) dataset file unloading and creation of Tensorflow Dataset

import h5py
import json
import tensorflow as tf
import numpy as np
from numpy import argwhere

# Open the dataset
hdf5_file = h5py.File("../dataset/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5",  'r')
# Load the modulation classes. You can also copy and paste the content of classes-fixed.txt.
modulation_classes = json.load(open("../dataset/2018.01/classes-fixed.json", 'r'))
#List groups of the hdf5 file
list(hdf5_file.keys())

# Read the HDF5 groups
# Dataset is to large for allocated RAM, so every fourth frame will be used, resulting in a fourth of the dataset being used
data = hdf5_file['X'][0:1277952:8]
modulation_onehot = hdf5_file['Y'][0:1277952:8]
snr = hdf5_file['Z'][0:1277952:8]

#creation of test dataset
test_data = data[1277952::128]
test_mod = modulation_onehot[1277952::128]
test_snr = snr[1277952::128]

# Closes the file
hdf5_file.close()

#converting from onehot to integer
mod_array = np.zeros(len(modulation_onehot))
i = 0
for x in modulation_onehot:
    mod_array[i] = np.where(x==1)[0][0]
    i = i + 1


    
#taking the three arrays and converting them to a tf dataset object
dataset = tf.data.Dataset.from_tensor_slices((data, mod_array))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_mod))

2021-10-21 20:14:05.451869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 20:14:05.451889: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-21 20:14:08.670363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-21 20:14:08.670383: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-21 20:14:08.670400: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bolt): /proc/driver/nvidia/version does not exist
2021-10-21 20:14:08.670599: I tensorflow/core/platform/cpu_feature_guard

In [2]:
#Tensorflow Dataset Object normalization, preprocessing, and customization block
batch_size = 1024
buf_size = 26624
drop_rem = True

dataset = dataset.shuffle(buffer_size=buf_size)
dataset = dataset.batch(batch_size, drop_remainder=drop_rem)

In [3]:
#model variables
img_width, img_height, img_channels = (1024, 2, 1)
opt = 'Adam'
lss = 'sparse_categorical_crossentropy'


model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(64, 3, activation='relu', 
                  input_shape=(img_width, img_height)),
      tf.keras.layers.MaxPooling1D(2, 2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(24, activation=tf.nn.softmax)
    ])

model.compile(optimizer=opt, loss=lss, metrics=['accuracy'])


In [4]:
model.fit(dataset, epochs=5)

2021-10-21 20:14:13.546887: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10% of free system memory.
2021-10-21 20:14:14.169045: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10% of free system memory.


Epoch 1/5


2021-10-21 20:14:15.109262: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-21 20:14:15.249475: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 267911168 exceeds 10% of free system memory.
2021-10-21 20:14:15.311422: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 133955584 exceeds 10% of free system memory.


156/156 [==============================] - 122s 782ms/step - loss: 2.0178 - accuracy: 0.3066
Epoch 2/5
156/156 [==============================] - 130s 832ms/step - loss: 1.6829 - accuracy: 0.3422
Epoch 3/5
156/156 [==============================] - 124s 797ms/step - loss: 1.5956 - accuracy: 0.3787
Epoch 4/5
156/156 [==============================] - 125s 803ms/step - loss: 1.5893 - accuracy: 0.3872
Epoch 5/5
156/156 [==============================] - 126s 806ms/step - loss: 1.5885 - accuracy: 0.3893


In [47]:
# Open the dataset
hdf5_file = h5py.File("../dataset/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5",  'r')

to_test = hdf5_file['X'][:1277953:16384]
to_test_label = hdf5_file['Y'][:1277953:16384]

#converting from onehot to integer
mod_array_test = np.zeros(len(to_test_label))
i = 0
for x in to_test_label:
    mod_array_test[i] = np.where(x==1)[0][0]
    i = i + 1

# Closes the file
hdf5_file.close()

#print(to_test)
print(mod_array_test)

[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.
  2.  2.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.
  5.  5.  5.  6.  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.  7.  7.  8.  8.
  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9. 10. 10. 10. 10. 10. 10. 10.
 11. 11. 11. 11. 11. 11. 12.]


In [48]:
print("Prediction on test data")
results = model.predict(to_test)
print("test loss, test acc:", results)

Prediction on test data
test loss, test acc: [[6.7485431e-03 5.6077703e-03 4.7836611e-03 ... 5.6958734e-04
  8.3384028e-04 9.8670786e-04]
 [6.0779038e-03 5.0213435e-03 4.2615305e-03 ... 4.8728369e-04
  7.1420718e-04 8.4916782e-04]
 [4.6819029e-03 4.0467381e-03 3.3613329e-03 ... 2.5977354e-04
  4.1184519e-04 4.9408939e-04]
 ...
 [2.2800094e-03 2.1928956e-03 1.6711229e-03 ... 2.5721449e-05
  4.5258461e-05 5.3723001e-05]
 [1.9068668e-03 1.7143006e-03 1.4685778e-03 ... 3.3857799e-05
  6.8288988e-05 7.1069611e-05]
 [6.2868539e-03 5.2415687e-03 4.4609006e-03 ... 5.2801531e-04
  7.8517059e-04 9.3135599e-04]]


In [60]:
predictions = [p.argmax() for p in results]

print(len(predictions))
print(*[(c, a, c == a) for c, a in zip(mod_array_test, predictions)], sep="\n")

79
(0.0, 11, False)
(0.0, 11, False)
(0.0, 11, False)
(0.0, 2, False)
(0.0, 2, False)
(0.0, 2, False)
(0.0, 2, False)
(1.0, 11, False)
(1.0, 11, False)
(1.0, 0, False)
(1.0, 2, False)
(1.0, 2, False)
(1.0, 2, False)
(2.0, 11, False)
(2.0, 11, False)
(2.0, 11, False)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 2, True)
(3.0, 11, False)
(3.0, 11, False)
(3.0, 11, False)
(3.0, 11, False)
(3.0, 11, False)
(3.0, 11, False)
(4.0, 11, False)
(4.0, 11, False)
(4.0, 11, False)
(4.0, 8, False)
(4.0, 7, False)
(4.0, 7, False)
(4.0, 7, False)
(5.0, 11, False)
(5.0, 11, False)
(5.0, 8, False)
(5.0, 7, False)
(5.0, 7, False)
(5.0, 7, False)
(6.0, 11, False)
(6.0, 11, False)
(6.0, 10, False)
(6.0, 7, False)
(6.0, 7, False)
(6.0, 7, False)
(6.0, 7, False)
(7.0, 11, False)
(7.0, 11, False)
(7.0, 8, False)
(7.0, 7, True)
(7.0, 8, False)
(7.0, 7, True)
(8.0, 11, False)
(8.0, 11, False)
(8.0, 11, False)
(8.0, 10, False)
(8.0, 8, True)
(8.0, 8, True)
(8.0, 8, True)
(9.0, 11, False)
(9.0, 11, False)
